In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

In [2]:
input = Path("example.txt").read_text()
print(input)

2333133121414131402



In [3]:
# Solution 1
fill = True
disk_map = []
id = 0
for size in input[:-1]:
    disk_map += [str(id) if fill else pd.NA] * int(size)
    id = id + (1 if fill else 0)
    fill = not fill
disk_map = pd.Series(disk_map, dtype="Int16")
disk_map

0        0
1        0
2     <NA>
3     <NA>
4     <NA>
5        1
6        1
7        1
8     <NA>
9     <NA>
10    <NA>
11       2
12    <NA>
13    <NA>
14    <NA>
15       3
16       3
17       3
18    <NA>
19       4
20       4
21    <NA>
22       5
23       5
24       5
25       5
26    <NA>
27       6
28       6
29       6
30       6
31    <NA>
32       7
33       7
34       7
35    <NA>
36       8
37       8
38       8
39       8
40       9
41       9
dtype: Int16

In [4]:
i = 0
free_index = disk_map[disk_map.isna()].index.min()
file_index = disk_map[disk_map.notna()].index.max()
while free_index < file_index:
    file = disk_map[file_index]
    disk_map[file_index] = pd.NA
    disk_map[free_index] = file
    free_index = disk_map[disk_map.isna()].index.min()
    file_index = disk_map[disk_map.notna()].index.max()
    i += 1
    if i % 1000 == 0:
        print(free_index, file_index)
    if i > 100000:
        break
print(i)

12


In [5]:
print(disk_map)

0        0
1        0
2        9
3        9
4        8
5        1
6        1
7        1
8        8
9        8
10       8
11       2
12       7
13       7
14       7
15       3
16       3
17       3
18       6
19       4
20       4
21       6
22       5
23       5
24       5
25       5
26       6
27       6
28    <NA>
29    <NA>
30    <NA>
31    <NA>
32    <NA>
33    <NA>
34    <NA>
35    <NA>
36    <NA>
37    <NA>
38    <NA>
39    <NA>
40    <NA>
41    <NA>
dtype: Int16


In [6]:
(disk_map[disk_map > 0] * disk_map[disk_map > 0].index).sum()

1928

In [7]:
# Solution 2

In [8]:
from dataclasses import dataclass


@dataclass
class Block:
    start: int
    stop: int
    index: int

    @property
    def size(self) -> int:
        return self.stop - self.start

    @property
    def is_fill(self) -> bool:
        return self.index != -1

    @property
    def is_free(self) -> bool:
        return self.index == -1

In [9]:
free = False
index = 0
start = 0
blocks = []
for size in input.strip():
    stop = start + int(size)
    block = Block(
        start=start,
        stop=stop,
        index=-1 if free else index,
    )
    blocks.append(block)
    start = stop
    if free:
        index += 1
    free = not free

blocks

[Block(start=0, stop=2, index=0),
 Block(start=2, stop=5, index=-1),
 Block(start=5, stop=8, index=1),
 Block(start=8, stop=11, index=-1),
 Block(start=11, stop=12, index=2),
 Block(start=12, stop=15, index=-1),
 Block(start=15, stop=18, index=3),
 Block(start=18, stop=19, index=-1),
 Block(start=19, stop=21, index=4),
 Block(start=21, stop=22, index=-1),
 Block(start=22, stop=26, index=5),
 Block(start=26, stop=27, index=-1),
 Block(start=27, stop=31, index=6),
 Block(start=31, stop=32, index=-1),
 Block(start=32, stop=35, index=7),
 Block(start=35, stop=36, index=-1),
 Block(start=36, stop=40, index=8),
 Block(start=40, stop=40, index=-1),
 Block(start=40, stop=42, index=9)]

In [10]:
num_blocks = len(blocks)
i = num_blocks - 1
while i > 0:
    j = 0
    moved = False
    while j < i:
        fill_block = blocks[i]
        free_block = blocks[j]
        fill_block_size = fill_block.size
        if (
            not fill_block.is_free
            and free_block.is_free
            and fill_block.size <= free_block.size
        ):

            fill_block.start = free_block.start
            fill_block.stop = fill_block.start + fill_block_size
            free_block.start += fill_block_size
            skip_empty = 1 if (free_block.size == 0) else 0
            blocks = (
                blocks[:j] + [fill_block] + blocks[j + skip_empty : i] + blocks[i + 1 :]
            )

            moved = True
            break
        j += 1
    if not moved:
        i -= 1
        if i % 1000 == 0:
            print(i)
blocks

0


[Block(start=0, stop=2, index=0),
 Block(start=2, stop=4, index=9),
 Block(start=4, stop=5, index=2),
 Block(start=5, stop=8, index=1),
 Block(start=8, stop=11, index=7),
 Block(start=12, stop=14, index=4),
 Block(start=14, stop=15, index=-1),
 Block(start=15, stop=18, index=3),
 Block(start=18, stop=19, index=-1),
 Block(start=21, stop=22, index=-1),
 Block(start=22, stop=26, index=5),
 Block(start=26, stop=27, index=-1),
 Block(start=27, stop=31, index=6),
 Block(start=31, stop=32, index=-1),
 Block(start=35, stop=36, index=-1),
 Block(start=36, stop=40, index=8),
 Block(start=40, stop=40, index=-1)]

In [11]:
df = pd.DataFrame(blocks)
df = df[df["index"] != -1].sort_values("start")
df

,start,stop,index
0,0,2,0
1,2,4,9
2,4,5,2
3,5,8,1
4,8,11,7
5,12,14,4
7,15,18,3
10,22,26,5
12,27,31,6
15,36,40,8


In [12]:
def score(x) -> int:
    return np.sum(np.arange(x["start"], x["stop"]) * x["index"])


df.apply(score, axis="columns").sum()

2858